In [1]:
from flask import Flask, render_template, request, redirect, url_for, send_from_directory, jsonify, send_file
from flask_socketio import SocketIO, emit

import os
import subprocess
import time

from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from werkzeug.utils import secure_filename
from wtforms.validators import InputRequired

import os, shutil, os.path

In [2]:
import pandas as pd
import numpy as np
from scipy import optimize
import random

from matplotlib import pyplot as plt
from matplotlib.figure import Figure

import matplotlib.ticker as ticker
import csv
import time
import integrator
import atm.core
import atm.jobs as jr
import re
import seaborn as sns

In [3]:
from io import BytesIO
import base64

### Main code

Here the main code is located. It is based on the article https://journals.aps.org/prb/abstract/10.1103/PhysRevB.108.184103
It cannot be published here, so I included only the example of sending graph to the website. 

In [6]:
class MainClass:
    def __init__(self, Tmelt, structure):
        # init function
    
    
    def Lines(self, Tmelt=None):
        x = np.linspace(0.6*Tmelt, 1.4*Tmelt, 1000)
        y = ksol*x+bsol
        t = kliq*x+bliq
        plt.plot(x,y, label='Potential energy of solid phase',  color='blue')
        plt.plot(x,t, label='Potential energy of liquid phase', color='indianred')
        plt.legend()
        plt.grid()
        plt.xlabel('Temperature, K')
        plt.ylabel('Potential energy, eV')
        plt.title('Al EAM')
        plt.savefig('lines.png', facecolor='white',dpi=200)

        # Send Graph to HTML
        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        plt.close()
        socketio.emit('show_graph', base64.b64encode(buffer.read()).decode())
        
        return ksol, bsol, kliq, bliq

    
    def MainFunc(self,name):
        Lines(...)
        self.WriteResults("Gaussian process finished")


### Main Flask Code

In [4]:
app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

socketio = SocketIO(app)


@socketio.on('build_graphs')
def handle_build_graphs(data):
    T = int(data['number'])
    lattice = data['option']
    potential = data['file']
    print(f'Someone pressed the button: {T}, {lattice}...')
    Alum = MainClass(T, lattice)
    Alum.MainFunc('Al_0707.csv')
    socketio.emit('finish')
    

@app.route('/try', methods=['GET', 'POST'])
def try_page():
    if request.method == 'POST':
        if 'file' in request.files:
            file = request.files['file']
            if file:
                path = os.path.join(app.config['UPLOAD_FOLDER'])
                fldr_to = f'user'
                #shutil.copytree(os.path.join(path, 'template'), os.path.join(path, fldr_to))
                filename = os.path.join(path, fldr_to, file.filename)
                file.save(filename)
                return "File uploaded"
    return render_template('index.html')


# Route for the main page
@app.route('/')
def main_page():
    return render_template('base.html')


if __name__ == '__main__':
    socketio.run(app, allow_unsafe_werkzeug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:52] "GET /static/images/side.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:52] "GET /static/images/arrow-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:52] "GET /static/images/punctir-left.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:52] "GET /static/images/punctir-right.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:53] "GET /static/images/background-2.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:53] "GET /static/images/cursor.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:53] "GET /static/images/fcc.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:53] "GET /static/images/track.gif HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 10:08:53] "GE

## Examples

### File Upload Example

In [ ]:
@app.route('/try', methods=['GET', 'POST'])
def try_page():
    if request.method == 'POST':
        if 'file' in request.files:
            file = request.files['file']
            if file:
                path = os.path.join(app.config['UPLOAD_FOLDER'])
                fldr_to = f'user'
                #shutil.copytree(os.path.join(path, 'template'), os.path.join(path, fldr_to))
                filename = os.path.join(path, fldr_to, file.filename)
                file.save(filename)
                return "File uploaded"
    return render_template('index.html')

### Display 5 graphs on the page

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import threading
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

def generate_graphs():
    graphs = []
    colors = ['red', 'orange', 'green', 'blue', 'purple']
    for i in range(5):  # Generate 5 graphs
        plt.figure()
        plt.plot(range(10), [x ** 2 for x in range(10)], color=colors[i])
        plt.xlabel('X-axis')
        plt.ylabel('Y-axis')
        plt.title(f'Graph {i + 1}')
        plt.grid(True)
        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        plt.close()
        time.sleep(2)
        # Send the finished message
        socketio.emit('converge_graph', base64.b64encode(buffer.read()).decode())
    socketio.emit('status', 'finished')

@app.route('/')
def index():
    return render_template('try-1.html')

@socketio.on('connect')
def handle_connect():
    # Start a new thread to generate graphs and send them to the client
    threading.Thread(target=generate_graphs).start()

if __name__ == '__main__':
    socketio.run(app, allow_unsafe_werkzeug=True)